При вставке/обновлении значений в таблице `Payment` срабатывает триггер `T_Payment_AI`. Внутри него используются несколько функций: `F_CalculatePaymentParticipantBalance`, `F_CalculateBalanceByMaterial`, `F_CalculateBalanceByWork`, `F_CalculateProjectBalance`. В этих функциях выполняются операции `SELECT` над полями таблиц, у которых изначально нет индекса, поэтому для таких полей можно добавить индексы:

    - NONCLUSTERED INDEX iName_PaymentCategory ON PaymentCategory (Name)
    - NONCLUSTERED INDEX iName_AccountType ON AccountType (Name)
    - NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON Supplier (ProfitByMaterialAsPayer)
    - NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON Supplier (ProfitByMaterialAsPayee)
    - NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON PaymentCategory (ProfitByMaterial)
    - NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON PaymentCategory (CostByMaterial)
    - NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON PaymentCategory (NotInPaymentParticipantProfit)
  

In [1]:
import pyodbc 

conn_str = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=PaymentData;'
    r'Trusted_Connection=yes;'
    r'Encrypt=no;'
    )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

Для начала проверим время исполнения запросов `INSERT` и `UPDATE` без использования новых индексов

In [2]:
def get_random_payment_participant_oid():
    query = """
    SELECT TOP 1 Oid
    FROM PaymentParticipant
    ORDER BY NEWID() 
    """
    oid = cursor.execute(query).fetchval()
    return oid

def get_random_payment_category_oid():
    query = """
    SELECT TOP 1 Oid
    FROM PaymentCategory
    ORDER BY NEWID() 
    """
    oid = cursor.execute(query).fetchval()
    return oid

def get_random_project_oid():
    query = """
    SELECT TOP 1 Oid
    FROM Project
    ORDER BY NEWID() 
    """
    oid = cursor.execute(query).fetchval()
    return oid

def get_random_payment_oid():
    query = """
    SELECT TOP 1 Oid
    FROM Payment
    ORDER BY NEWID() 
    """
    oid = cursor.execute(query).fetchval()
    return oid

In [3]:
from random import randint

def random_insert(n_rows):
    for _ in range(n_rows):
        payer_oid = get_random_payment_participant_oid()
        payee_oid = get_random_payment_participant_oid() 
        payment_category_oid = get_random_payment_category_oid()
        project_oid = get_random_project_oid()

        query = f"""
        INSERT INTO Payment
        (Oid, Amount, Category, Project, Justification,
        Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
        CreateDate, CheckNumber, IsAuthorized, Number)
        VALUES
        (NEWID(), '{randint(0, 10000)}', '{payment_category_oid}', '{project_oid}', NULL,
        'Random inserted payment', NULL, '{payer_oid}', '{payee_oid}', NULL, NULL,
        NULL, NULL, NULL, NULL)
        """
        cursor.execute(query)
    cursor.commit()

def random_update(n_rows):
    for _ in range(n_rows):
        payer_oid = get_random_payment_participant_oid()
        payee_oid = get_random_payment_participant_oid() 
        payment_category_oid = get_random_payment_category_oid()
        payment_oid = get_random_payment_oid()

        query = f"""
        UPDATE Payment
        SET Amount = Amount + 42, Payer = '{payer_oid}', Payee = '{payee_oid}', Category = '{payment_category_oid}'
        WHERE Oid = '{payment_oid}'
        """
        cursor.execute(query)
    cursor.commit()

In [4]:
%%timeit -r 1 -n 1

random_insert(500)

2.77 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -r 1 -n 1

random_update(500)

2.36 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Добавим индексы и сравним время

In [6]:
indexes = [
    "NONCLUSTERED INDEX iName_PaymentCategory ON PaymentCategory (Name)",
    "NONCLUSTERED INDEX iName_AccountType ON AccountType (Name)",
    "NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON Supplier (ProfitByMaterialAsPayer)",
    "NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON Supplier (ProfitByMaterialAsPayee)",
    "NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON PaymentCategory (ProfitByMaterial)",
    "NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON PaymentCategory (CostByMaterial)",
    "NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON PaymentCategory (NotInPaymentParticipantProfit)",
]

In [7]:
for index in indexes:
    query = f"""
    CREATE {index}
    """
    cursor.execute(query)
    cursor.commit()

In [8]:
truncate_payment = """
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[Payment]') AND type in (N'U'))
TRUNCATE TABLE [dbo].[Payment]
"""
cursor.execute(truncate_payment)
cursor.commit()

In [9]:
%%timeit -r 1 -n 1

random_insert(500)

2.86 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [10]:
%%timeit -r 1 -n 1

random_update(500)

2.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Были протестированы различные комбинации создания новых индексов, но ни один из них не дал никакого улучшения исполнения времени вставки/изменения.